In [1]:
import pandas as pd
import numpy as np
import re
import glob
import os
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer
from emot.emo_unicode import UNICODE_EMOJI
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import nltk
import gensim
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
import seaborn as sns
#nltk.download('punkt')

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV

%matplotlib inline

/Users/priyankamanam/opt/anaconda3/lib/python3.9/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/Users/priyankamanam/opt/anaconda3/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [2]:
data_files = r"Tweets"
team_tweets_df= pd.DataFrame()
team_tweets_df= pd.concat(map(pd.read_csv,glob.glob(os.path.join(data_files,'Team*.csv'))), ignore_index= True)
team_tweets_df.drop_duplicates(inplace=True)

In [3]:
# team_tweets_df = pd.read_csv("Team_tweets_17_03_2020.csv")
# team_tweets_df.shape
team_tweets_df.head(10)

,Team,Date,Tweet
0,Boston Bruins,2022-03-24 18:13:29+00:00,RT @SassTreasures: Check out this listing I ju...
1,Boston Bruins,2022-03-24 18:10:39+00:00,RT @tmlfaninvan: the Toronto Maple Leafs colla...
2,Boston Bruins,2022-03-24 18:10:03+00:00,Check out this listing I just added to my #Pos...
3,Boston Bruins,2022-03-24 18:02:10+00:00,@LeafsLindsay @ClendelWark Win for us is Tampa...
4,Boston Bruins,2022-03-24 18:01:46+00:00,Today's NHL Picks: 7:00 PM ET - Tampa Bay Ligh...
5,Boston Bruins,2022-03-24 18:00:20+00:00,Just one point separates the Boston Bruins and...
6,Boston Bruins,2022-03-24 17:48:25+00:00,RT @BOSTONSPORTSB: Boston Sports rankings (cur...
7,Boston Bruins,2022-03-24 17:48:07+00:00,RT @BOSTONSPORTSB: Boston Sports rankings (cur...
8,Boston Bruins,2022-03-24 17:44:07+00:00,RT @cupofchowdah: Hampus Lindholm makes his Br...
9,Boston Bruins,2022-03-24 17:41:12+00:00,Hampus Lindholm makes his Bruins’ debut tonigh...


# Tweets cleaning

In [4]:
#removing hyoerlinks
team_tweets_df["Tweet"] = team_tweets_df["Tweet"].str.replace(r'https?:\/\/.*[\r\n]*', '').str.strip()

#remove # and @ from tweets
team_tweets_df["Tweet"] = team_tweets_df["Tweet"].str.replace(r'@\w+|#\w+|RT|', '').str.strip()

/var/folders/sf/sxg31ljs3ks0h1bsflmz2z1c0000gn/T/ipykernel_1917/1076732081.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  team_tweets_df["Tweet"] = team_tweets_df["Tweet"].str.replace(r'https?:\/\/.*[\r\n]*', '').str.strip()
/var/folders/sf/sxg31ljs3ks0h1bsflmz2z1c0000gn/T/ipykernel_1917/1076732081.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  team_tweets_df["Tweet"] = team_tweets_df["Tweet"].str.replace(r'@\w+|#\w+|RT|', '').str.strip()


In [5]:
# Function for converting emojis into word
def handle_emojis(tweet):
    for emot in UNICODE_EMOJI:
        #tweet = tweet.replace(emot, (" " + "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split())))
        tweet = tweet.replace(emot, "")
    return tweet

team_tweets_df["Tweet"] = team_tweets_df["Tweet"].apply(handle_emojis)

# Sentiment Stats of each team

In [6]:
import time
team_names = team_tweets_df["Team"].unique()
analyzer = SentimentIntensityAnalyzer()
pd.options.mode.chained_assignment = None

Team_arr = []
positive_scores = []
negative_scores = []
neutral_scores = []

def determine_polarity(tweet):
    pol_scores = analyzer.polarity_scores(tweet) 
    compound_score = pol_scores["compound"]
    if compound_score >= 0.5:
        return 1
    elif compound_score <= -0.5:
        return -1
    else:
        return 0

train_test_df = pd.DataFrame()
train_df = pd.DataFrame()
test_df = pd.DataFrame()
pos_df = pd.DataFrame()
neg_df = pd.DataFrame()
neu_df = pd.DataFrame()

for team in team_names:
    curr_team_df = team_tweets_df.loc[team_tweets_df["Team"] == team]
    curr_team_df["polarity"] = curr_team_df["Tweet"].apply(determine_polarity)
    pos_df = pos_df.append(curr_team_df[curr_team_df["polarity"]== 1])
    neg_df = neg_df.append(curr_team_df[curr_team_df["polarity"]== -1])
    neu_df = neu_df.append(curr_team_df[curr_team_df["polarity"]== 0])

train_df = train_df.append(pos_df.iloc[0:600, :])    
train_df = train_df.append(neg_df.iloc[0:600, :])    
train_df = train_df.append(neu_df.iloc[0:600, :]) 

test_df = test_df.append(pos_df.iloc[600:, :])    
test_df = test_df.append(neg_df.iloc[600:, :])    
test_df = test_df.append(neu_df.iloc[600:, :]) 

print(train_df.shape)
print(test_df.shape)
#     pos_num = curr_team_df[curr_team_df["polarity"]== 1].count()["Tweet"]
#     neg_num = curr_team_df[curr_team_df["polarity"]== -1].count()["Tweet"]
#     neu_num = curr_team_df[curr_team_df["polarity"]== 0].count()["Tweet"]
    #curr_team_df[curr_team_df["polarity"]== "negative"].to_csv("checking_neg.csv")
    #time.sleep(1000)
#     Team_arr.append(team)
#     positive_scores.append(pos_num)
#     negative_scores.append(neg_num)
#     neutral_scores.append(neu_num)
    
#     print("\n=========== ",team ," Tweets Sentiment Stats: =================")
#     print("Positive Tweets: ", pos_num)
#     print("Negative Tweets: ", neg_num)
#     print("Neutral Tweets:  ", neu_num)

# Team_sentiment_stats_df = pd.DataFrame({"Team" : Team_arr,
#                               "Positive_tweet" : positive_scores,
#                               "Negative_tweet" : negative_scores,
#                               "Neutral_tweet" : neutral_scores
#                                        })
# print(Team_sentiment_stats_df)

# Team_sentiment_stats_df.to_csv('Teams_sentiment.csv')

(1800, 4)
(70286, 4)


In [7]:
#!conda install fastai -c fastai -c pytorch

In [8]:
from fastai.text import *
from fastai.text.learner import language_model_learner
tweet_model = language_model_learner(train_df, AWD_LSTM, drop_mult=0.3)

/Users/priyankamanam/opt/anaconda3/lib/python3.9/site-packages/tables/req_versions.py:20: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  min_numpy_version = LooseVersion('1.9.3')
/Users/priyankamanam/opt/anaconda3/lib/python3.9/site-packages/tables/req_versions.py:21: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  min_numexpr_version = LooseVersion('2.6.2')
/Users/priyankamanam/opt/anaconda3/lib/python3.9/site-packages/tables/req_versions.py:22: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  min_hdf5_version = LooseVersion('1.8.4')
/Users/priyankamanam/opt/anaconda3/lib/python3.9/site-packages/tables/req_versions.py:23: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  min_blosc_version = LooseVersion("1.4.1")
/Users/priyankamanam/opt/anaconda3/lib/python3.9/site-packages/tables/req_versions.py:24: D

NameError: name 'AWD_LSTM' is not defined

In [ ]:
font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 14,
        }

team_total = range(32)
fig = plt.figure(figsize = (10,6))
ax = plt.subplot(111)
plt.xlabel('Teams')
plt.ylabel('Positive / Negative Tweets')

#print(len(Team_sentiment_stats_df["Positive_tweet"].tolist()))
#print(len(Team_sentiment_stats_df["Team"].tolist()))
#print(len(Team_sentiment_stats_df["Negative_tweet"].tolist()))

n, bins, patches = plt.hist(Team_sentiment_stats_df["Team"].tolist(), Team_sentiment_stats_df["Team"].unique().__len__(), stacked=True, density=False)
plt.title("Sentiments Both in same direction", fontdict= font, loc='center')
#print(bins)
plt.xticks(ticks = bins[1:], labels = Team_sentiment_stats_df["Team"], rotation = 90, horizontalalignment = 'left')
ax.bar(team_total, Team_sentiment_stats_df["Positive_tweet"].tolist(), width=1,alpha=1, color='b',facecolor='blue', edgecolor='black')
ax.bar(team_total, Team_sentiment_stats_df["Negative_tweet"].tolist(), width=1, alpha=0.4,color='r',facecolor='red', edgecolor='black')
plt.show()
#plt.hist(x, density=False)  # density=False would make counts
#plt.ylabel('Probability')
#plt.xlabel('Data')

fig = plt.figure(figsize = (10,6))
ax = plt.subplot(111)
plt.xlabel('Teams')
plt.ylabel('Positive / Negative Tweets')

n, bins, patches = plt.hist(Team_sentiment_stats_df["Team"].tolist(), Team_sentiment_stats_df["Team"].unique().__len__(), stacked=True, density=False)
negative_data_opposite = [i * -1 for i in Team_sentiment_stats_df["Negative_tweet"].tolist()]

plt.xticks(ticks=bins[1:], labels=Team_sentiment_stats_df["Team"], rotation=90, horizontalalignment='left')
plt.title("Sentiments Negative Tweets in opposite direction", fontdict= font, loc='center')
ax.bar(team_total, Team_sentiment_stats_df["Positive_tweet"].tolist(), width=1, color='b',facecolor='blue', edgecolor='black')
ax.bar(team_total, negative_data_opposite, alpha=1, width=1, color='r', facecolor='red', edgecolor='black')

plt.show()

In [ ]:
#testing
"""
pos_tweets = [('I love this you car @hello', 'positive'),
    ('This view is amazing', 'positive'),
    ('I feel great this morning', 'positive'),
    ('I am so excited #1234 about the concert', 'positive'),
    ('He is my best friend', 'positive')]

test = pd.DataFrame(pos_tweets)
test.columns = ["tweet","class"]

test['tweet'] = test['tweet'].apply(stop_word_removal)
print(test)
"""

# Topic Modelling

In [ ]:
#remove punctuation
team_tweets_df["Tweet"] = team_tweets_df["Tweet"].str.replace('[{}]'.format(string.punctuation), '')

In [ ]:
#remove stop words

stop_wrds = stopwords.words('english')

def stop_word_removal(tweet):
    return ' '.join([word for word in tweet.split() if word not in stop_wrds])

team_tweets_df["Tweet"] = team_tweets_df["Tweet"].apply(stop_word_removal)

In [ ]:
#tokenization and stemming
##need to change it to lemmatization
tokenizer = TweetTokenizer(
    preserve_case=False,
    strip_handles=True,
    reduce_len=True)

snowball_stemmer = SnowballStemmer("english")

def tokenization_stemming(tweet):
    sentence_token_stem = []
    token_sent = tokenizer.tokenize(tweet)
    for token in token_sent:
        sentence_token_stem.append(snowball_stemmer.stem(token))
    #disabled stemming
    return token_sent

team_tweets_df["Tweet"] = team_tweets_df["Tweet"].apply(tokenization_stemming)
# team_tweets_df.to_csv('check.csv', index = False)

In [ ]:
#lemmatization
import spacy 

# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
# trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)
def remove_team_name(tweet):
    split_team = str.lower(sample_team_name).split()
    split_team.append(split_team[-1][:-1])
    #print(split_team)
    return ' '.join([word for word in tweet if word not in split_team])
    
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    #bigram = gensim.models.Phrases(texts, min_count=2, threshold=10) # higher threshold fewer phrases.
    #trigram = gensim.models.Phrases(bigram[texts], min_count=3, threshold=10)  
    #bigram_mod = gensim.models.phrases.Phraser(bigram)
    #trigram_mod = gensim.models.phrases.Phraser(trigram)
    #texts_out_bi = [bigram_mod[doc] for doc in texts]
    split_team = str.lower(sample_team_name).split()
    split_team.append(split_team[-1][:-1])
    #print(texts_out_bi)
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        #texts_out.append([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags])
        texts_out.append(" ".join([token.lemma_ for token in doc if (token.pos_ in allowed_postags) and (token.lemma_ not in split_team)]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en

nlp = spacy.load("en_core_web_sm")

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
sample_team_name = "Boston Bruins"
sample_team_df =  team_tweets_df.loc[team_tweets_df["Team"] == sample_team_name]
#sample_team_df.head(5)
#sample_team_df["Tweet"] = sample_team_df["Tweet"].apply(remove_team_name)
sample_team_df["Tweet"] = lemmatization(sample_team_df["Tweet"], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#team_tweets_df["Tweet"] = team_tweets_df["Tweet"].apply(tokenization_stemming)
#print(data_lemmatized[:2])
sample_team_df.head(5)

In [ ]:
#LDA Topic Modelling
#needs improvement
#sample_team_df =  team_tweets_df.loc[team_tweets_df["Team"] == "Columbus Blue Jackets"]
# tokenized_tweets = sample_team_df["Tweet"].tolist()
# dictionary = corpora.Dictionary(tokenized_tweets)
# doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_tweets]


# LDA = gensim.models.ldamodel.LdaModel

# lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=3, random_state=100, chunksize=100, passes=50,iterations=100) 


In [ ]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             #ngram_range=(1)
                             #token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(sample_team_df["Tweet"])

In [ ]:
#Checking the sparsity of the data
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("sparsity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

In [ ]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

In [ ]:
# Get Log Likelyhoods from Grid Search Output
n_topics = [10, 15, 20, 25, 30]
#print(model.cv_results_)
scores_mean = model.cv_results_['mean_test_score'].tolist()
print(scores_mean)
log_likelyhoods_5 = scores_mean[0:5]
log_likelyhoods_7 = scores_mean[5:10]
log_likelyhoods_9 = scores_mean[10:15]

# Show graph
plt.figure(figsize=(12, 8))
plt.plot(n_topics, log_likelyhoods_5, label='0.5')
plt.plot(n_topics, log_likelyhoods_7, label='0.7')
plt.plot(n_topics, log_likelyhoods_9, label='0.9')
plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

In [ ]:
# pyLDAvis.enable_notebook()
# vis = gensimvis.prepare(best_lda_model, data_vectorized, vectorizer)
# vis 

In [ ]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

In [ ]:
def show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

In [ ]:
from sklearn.cluster import KMeans
lda_output = best_lda_model.transform(data_vectorized)
svd_model = TruncatedSVD(n_components=2)  # 2 components
lda_output_svd = svd_model.fit_transform(lda_output)

# X and Y axes of the plot using SVD decomposition
x = lda_output_svd[:, 0]
y = lda_output_svd[:, 1]
clusters = KMeans(n_clusters=10, random_state=100).fit_predict(lda_output)
plt.figure(figsize=(12, 12))
plt.scatter(x, y, c=clusters)
plt.xlabel('Component 2')
plt.xlabel('Component 1')
plt.title("Segregation of Topic Clusters", )